# Prepare dataset

- Create dataset from the new dataframe

## Librairies

In [1]:
!pip install pandas==1.3.4
!pip install transformers==4.12.5
!pip install datasets==1.15.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import os
import pickle

import pandas as pd

import numpy as np
import torch

import transformers
from transformers import BertTokenizer

import datasets
from datasets import concatenate_datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import DatasetDict

In [3]:
print('pandas:\t\t', pd.__version__)
print('transformers:\t', transformers.__version__)
print('datasets:\t', datasets.__version__)

pandas:		 1.3.4
transformers:	 4.12.5
datasets:	 1.15.1


## Load Data

In [4]:
dataset_df = pd.read_pickle("/notebooks/Prompting/notebooks/pe_dataset_w_prompts_1_pickle")

In [5]:
dataset_df

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,nr_preceeding_comps_in_para,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_1
0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
1,essay001,T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,1.00,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADV, ADJ, NOUN, ADP,...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
2,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,0,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
3,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,1,2,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
4,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,2,1,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5968,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,4,3,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying har...
5969,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,5,2,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, VERB, NOUN, VE...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying har...
5970,essay402,T13,Premise 1393 1436,they can contribute positively to community,Premise,Premise,[],Support,Linked,TRAIN,...,6,1,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: PRON, VERB, VERB, ADV, AD...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying ha

## Prepare dataframe

In [6]:
dataset_df.columns

Index(['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_x',
       'label_ComponentType', 'relation_SupportAttack', 'label_RelationType',
       'label_LinkedNotLinked', 'split', 'essay', 'argument_bound_1',
       'argument_bound_2', 'argument_id', 'sentence', 'paragraph', 'para_nr',
       'total_paras', 'token_count', 'token_count_covering_para',
       'tokens_count_covering_sentence', 'preceeding_tokens_in_sentence_count',
       'succeeding_tokens_in_sentence_count', 'token_ratio',
       'relative_position_in_para_char', 'is_in_intro',
       'relative_position_in_para_token', 'is_in_conclusion',
       'is_first_in_para', 'is_last_in_para', 'nr_preceeding_comps_in_para',
       'nr_following_comps_in_para', 'structural_fts_as_text',
       'structural_fts_as_text_combined', 'para_ratio', 'first_or_last',
       'strct_fts_w_position_in_essay', 'component_pos_tags',
       'strct_fts_essay_position_pos_tags', 'prompted_representation_1'],
      dtype='object')

In [7]:
# sanity check
print(len(dataset_df))
dataset_df = dataset_df.dropna()
print(len(dataset_df))

5973
5973


In [8]:
dataset_df = dataset_df.reset_index(drop=True)

In [9]:
dataset_df

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,nr_preceeding_comps_in_para,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_1
0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
1,essay001,T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,1.00,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADV, ADJ, NOUN, ADP,...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
2,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,0,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
3,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,1,2,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
4,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,2,1,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5968,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,4,3,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying har...
5969,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,5,2,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, VERB, NOUN, VE...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying har...
5970,essay402,T13,Premise 1393 1436,they can contribute positively to community,Premise,Premise,[],Support,Linked,TRAIN,...,6,1,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: PRON, VERB, VERB, ADV, AD...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying ha

In [10]:
dataset_df = dataset_df.rename(columns={"split_y": "split"})

In [11]:
dataset_df.columns

Index(['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_x',
       'label_ComponentType', 'relation_SupportAttack', 'label_RelationType',
       'label_LinkedNotLinked', 'split', 'essay', 'argument_bound_1',
       'argument_bound_2', 'argument_id', 'sentence', 'paragraph', 'para_nr',
       'total_paras', 'token_count', 'token_count_covering_para',
       'tokens_count_covering_sentence', 'preceeding_tokens_in_sentence_count',
       'succeeding_tokens_in_sentence_count', 'token_ratio',
       'relative_position_in_para_char', 'is_in_intro',
       'relative_position_in_para_token', 'is_in_conclusion',
       'is_first_in_para', 'is_last_in_para', 'nr_preceeding_comps_in_para',
       'nr_following_comps_in_para', 'structural_fts_as_text',
       'structural_fts_as_text_combined', 'para_ratio', 'first_or_last',
       'strct_fts_w_position_in_essay', 'component_pos_tags',
       'strct_fts_essay_position_pos_tags', 'prompted_representation_1'],
      dtype='object')

## train_test_split

In [12]:
train_df = dataset_df[dataset_df.split=='TRAIN'].reset_index(drop=True)
test_df = dataset_df[dataset_df.split=='TEST'].reset_index(drop=True)



# for example

# val_df = dataset_df[dataset_df.split=='VALIDATION'].reset_index(drop=True)

## Prepare Dataset object

In [13]:
dataset_train = Dataset.from_pandas(train_df)
dataset_test = Dataset.from_pandas(test_df)

In [14]:
train_val_datasets = dataset_train.train_test_split(train_size=0.8)
dataset_train = train_val_datasets['train']
dataset_val = train_val_datasets['test']

In [15]:
dataset = DatasetDict({"train": dataset_train, "test": dataset_test, "validation": dataset_val})

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_x', 'label_ComponentType', 'relation_SupportAttack', 'label_RelationType', 'label_LinkedNotLinked', 'split', 'essay', 'argument_bound_1', 'argument_bound_2', 'argument_id', 'sentence', 'paragraph', 'para_nr', 'total_paras', 'token_count', 'token_count_covering_para', 'tokens_count_covering_sentence', 'preceeding_tokens_in_sentence_count', 'succeeding_tokens_in_sentence_count', 'token_ratio', 'relative_position_in_para_char', 'is_in_intro', 'relative_position_in_para_token', 'is_in_conclusion', 'is_first_in_para', 'is_last_in_para', 'nr_preceeding_comps_in_para', 'nr_following_comps_in_para', 'structural_fts_as_text', 'structural_fts_as_text_combined', 'para_ratio', 'first_or_last', 'strct_fts_w_position_in_essay', 'component_pos_tags', 'strct_fts_essay_position_pos_tags', 'prompted_representation_1'],
        num_rows: 3770
    })
    test: Dataset({
        features:

In [17]:
# sanity check
set(dataset['train']['split']), set(dataset['test']['split']), set(dataset['validation']['split'])

({'TRAIN'}, {'TEST'}, {'TRAIN'})

## Save dataset

In [19]:
torch.save(dataset, 'pe_dataset_w_prompts_1.pt')